In [20]:
#import the module
import sys
sys.path.append(r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\scripts')
import eda_analysis as eda

# Load the Default Estimator

In [21]:
file_path = r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\data\DF_data.csv'
data = eda.data_loader(file_path)
non_numerical_columns = data.select_dtypes(include=['object', 'category']).columns
print(non_numerical_columns)


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'ProviderId', 'ProductId', 'ChannelId', 'TransactionStartTime',
       'TransactionDate', 'RFMS_Label'],
      dtype='object')


# Split the Data

In [14]:
# Exclude 'TransactionId' and other identifier columns from the features
X = data.drop(columns=['TransactionId', 'RFMS_Label', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'ProviderId', 'ProductId', 'ChannelId', 'TransactionStartTime',
       'TransactionDate', 'RFMS_Label'])  # Features
y = data['RFMS_Label']  # Target

# Convert target to numerical format
y = y.map({'good': 1, 'bad': 0})


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

# Fit and transform numerical columns
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scaling the Data

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit and transform the scaled data
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)


In [18]:
import joblib

# Save the scaler and imputer
joblib.dump(scaler, r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\scaler.pkl')
joblib.dump(imputer, r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\imputer.pkl')


['C:\\Users\\user\\Desktop\\Kifiya\\Bati_Bank_Credit_Scoring\\imputer.pkl']

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

# Assume `X_train_selected` contains only the selected features
X_train_selected = X_train[['Amount', 'Value', 'Frequency', 'Monetary', 'Recency']]

imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

# Fit the imputer and scaler on the selected features
X_train_imputed = imputer.fit_transform(X_train_selected)
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Save the imputer and scaler
joblib.dump(imputer, r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\imputer_selected.pkl')
joblib.dump(scaler, r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\scaler_selected.pkl')


['C:\\Users\\user\\Desktop\\Kifiya\\Bati_Bank_Credit_Scoring\\scaler_selected.pkl']

# Train the Models using Logistic Regression and Gradient Boosting model

In [14]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000)

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the Gradient Boosting model
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train_scaled, y_train)


GradientBoostingClassifier(random_state=42)

In [10]:
from sklearn.model_selection import cross_val_score

# Cross-validation using the scaled data for Gradient Boosting
gbm_cv_scores = cross_val_score(gbm, scaler.fit_transform(imputer.fit_transform(X)), y, cv=5)
print(f"Gradient Boosting Cross-Validation Scores: {gbm_cv_scores}")
print(f"Mean Gradient Boosting Cross-Validation Score: {gbm_cv_scores.mean()}")


Gradient Boosting Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean Gradient Boosting Cross-Validation Score: 1.0


In [11]:
from sklearn.model_selection import cross_val_score

# Cross-validation using the scaled data
log_reg_cv_scores = cross_val_score(log_reg, scaler.fit_transform(imputer.fit_transform(X)), y, cv=5)
print(f"Logistic Regression Cross-Validation Scores: {log_reg_cv_scores}")
print(f"Mean Logistic Regression Cross-Validation Score: {log_reg_cv_scores.mean()}")

Logistic Regression Cross-Validation Scores: [0.97951184 0.99231694 0.96257579 0.99649801 0.99482542]
Mean Logistic Regression Cross-Validation Score: 0.9851456007863527


# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Define parameter grid for solvers that support 'l2' penalty
param_dist_l2 = {
    'C': uniform(0.01, 10),  # Inverse of regularization strength
    'penalty': ['l2'],  # Regularization type
    'solver': ['lbfgs', 'saga'],  # Solvers that support 'l2' penalty
    'max_iter': [500, 1000, 1500],  # Increased number of iterations
    'tol': [1e-4, 1e-3, 1e-2]  # Increased tolerance
}

# Define parameter grid for solvers that support 'l1' penalty
param_dist_l1 = {
    'C': uniform(0.01, 10),  # Inverse of regularization strength
    'penalty': ['l1'],  # Regularization type
    'solver': ['liblinear', 'saga'],  # Solvers that support 'l1' penalty
    'max_iter': [500, 1000, 1500],  # Increased number of iterations
    'tol': [1e-4, 1e-3, 1e-2]  # Increased tolerance
}

# Combine the parameter grids
param_dist_combined = [param_dist_l2, param_dist_l1]

# Initialize RandomizedSearchCV for Logistic Regression
random_search = RandomizedSearchCV(estimator=LogisticRegression(), param_distributions=param_dist_combined, n_iter=100, cv=5, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Best Logistic Regression model
best_log_reg_model = random_search.best_estimator_


In [ ]:
from sklearn.model_selection import cross_val_score

# Cross-validation using the scaled data for the best Logistic Regression model
log_reg_cv_scores = cross_val_score(best_log_reg_model, scaler.fit_transform(imputer.fit_transform(X)), y, cv=5)
print(f"Best Logistic Regression Cross-Validation Scores: {log_reg_cv_scores}")
print(f"Mean Best Logistic Regression Cross-Validation Score: {log_reg_cv_scores.mean()}")


# Model Evaluation

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict on test data
y_pred = log_reg.predict(X_test_scaled)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(y_test, log_reg.predict_proba(X_test_scaled)[:, 1])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC-AUC: {roc_auc}")


Accuracy: 0.9936235822923745
Precision: 0.9969302424049963
Recall: 0.9902218483860793
F1 Score: 0.9935647220170905
ROC-AUC: 0.9999083864451189


In [15]:
import joblib
# Save the trained model
joblib.dump(log_reg, r'C:\Users\user\Desktop\Kifiya\Bati_Bank_Credit_Scoring\models\logistic_regression_model.pkl')


['C:\\Users\\user\\Desktop\\Kifiya\\Bati_Bank_Credit_Scoring\\logistic_regression_model.pkl']